In [ ]:
import pandas as pd

df = pd.read_csv('/content/dataset.csv')
df.head()


In [ ]:
df.shape
df.info()
df['disease'].value_counts()




In [ ]:
X = df.drop('disease', axis=1)
y = df['disease']
X.shape, y.shape



In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(y)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:
X = df.drop('disease', axis=1)

X = X.dropna(axis=1, how='all')

y = df['disease']


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)


In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='most_frequent')
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:
X_train.isnull().sum().sum(), X_test.isnull().sum().sum()


In [ ]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()
nb.fit(X_train, y_train)
nb_pred = nb.predict(X_test)



In [ ]:
nb_pred = nb.predict(X_test)


In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)
dt_pred = dt.predict(X_test)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)


In [ ]:
from sklearn.metrics import accuracy_score

print("Naive Bayes Accuracy:", accuracy_score(y_test, nb_pred))
print("Decision Tree Accuracy:", accuracy_score(y_test, dt_pred))
print("Random Forest Accuracy:", accuracy_score(y_test, rf_pred))


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

print("Naive Bayes Report\n", classification_report(y_test, nb_pred))
print("Decision Tree Report\n", classification_report(y_test, dt_pred))
print("Random Forest Report\n", classification_report(y_test, rf_pred))


In [ ]:
from sklearn.metrics import f1_score

print("Naive Bayes Dice:", f1_score(y_test, nb_pred, average='weighted'))
print("Decision Tree Dice:", f1_score(y_test, dt_pred, average='weighted'))
print("Random Forest Dice:", f1_score(y_test, rf_pred, average='weighted'))


In [ ]:
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

y_test_bin = label_binarize(y_test, classes=range(len(set(y))))
rf_prob = rf.predict_proba(X_test)

fpr, tpr, _ = roc_curve(y_test_bin.ravel(), rf_prob.ravel())
roc_auc = auc(fpr, tpr)

plt.plot(fpr, tpr)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve - Random Forest")
plt.show()

print("AUC:", roc_auc)


In [ ]:
def predict_disease(symptom_list):
    input_data = {col:0 for col in X.columns}
    for symptom in symptom_list:
        if symptom in input_data:
            input_data[symptom] = 1

    input_df = pd.DataFrame([input_data])
    prediction = rf.predict(input_df)[0]
    disease_name = le.inverse_transform([prediction])[0]
    confidence = max(rf.predict_proba(input_df)[0])

    return disease_name, confidence


In [ ]:
#trial


required_objects = ['df', 'X', 'rf', 'le', 'predict_disease']
missing = [obj for obj in required_objects if obj not in globals()]

if missing:
    print("⚠️ Please run all cells above before using the demo.")
    print("Missing:", missing)
else:
    print("Enter symptoms from the following list:")
    print(list(X.columns))

    user_symptoms = input("Enter symptoms separated by commas: ").lower().split(",")
    user_symptoms = [s.strip() for s in user_symptoms]

    disease, confidence = predict_disease(user_symptoms)

    print("\n🧠 Predicted Disease:", disease)
    print("📊 Confidence:", round(confidence * 100, 2), "%")


